In [ ]:
!pip install geemap rasterio geopandas folium flask-ngrok pyngrok geojson

import ee
from pyngrok import ngrok
ngrok.set_auth_token("your_auth_token")
!ngrok config add-authtoken 2zE9NVx5TK0gOrEKEWd7cQKRvWp_yq4ENU6Eht9t7TPbNTs

key_path = 'json file path in colab environment'
ee.Authenticate()
ee.Initialize(project='your_project_id')

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
TEMPL = """
<!DOCTYPE html>
<html>
<head>
  <title>Trace</title>
  <style>
    body {
      font-family: 'Lato', sans-serif;
      background-color: #f4f7f6;
      margin: 0;
      padding: 30px;
      color: #333;
    }

    h1 {
      text-align: center;
      color: #2e7d32;
      font-size: 3rem;
      line-height: 1.2;
      margin-bottom: 20px;
    }

    form {
      background: #ffffff;
      padding: 20px;
      border-radius: 8px;
      box-shadow: 0 2px 8px rgba(0,0,0,0.1);
      max-width: 700px;
      margin: 0 auto 30px auto;
    }

    form input[type="text"],
    form input[type="date"],
    form select {
      padding: 8px;
      margin: 10px 5px;
      border: 1px solid #ccc;
      border-radius: 4px;
      font-size: 14px;
    }

    form input[type="submit"] {
      background-color: #2e7d32;
      color: white;
      padding: 10px 20px;
      border: none;
      border-radius: 4px;
      cursor: pointer;
      font-weight: bold;
    }

    form input[type="submit"]:hover {
      background-color: #27642a;
    }

    .map-container {
      margin-bottom: 40px;
    }

    .content-wrapper {
      display: flex;
      justify-content: center;
      align-items: flex-start;
      gap: 40px;
      flex-wrap: wrap;
    }

    .card {
      background: #ffffff;
      padding: 20px;
      border-radius: 8px;
      box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    }

    #imageMode img,
    #mapMode iframe {
      width: 100%;
      max-width: 450px;
      border-radius: 4px;
    }

    #slider {
      width: 100%;
      max-width: 450px;
      margin-top: 10px;
    }

    #frame_date {
      font-family: 'Lato', sans-serif;
      font-size: 14px;
      color: #ccc;
      margin-top: 8px;
    }

    #mapMode {
      display: none;
      width: 450px;
      height: 450px;
      overflow: hidden;
      position: relative;
    }

    #mapMode .folium-map {
      width: 450px !important;
      height: 450px !important;
      max-width: 100%;
      max-height: 100%;
      margin: 0 auto;
    }

    #mapMode iframe {
      width: 450px !important;
      height: 450px !important;
      border: none;
    }

    button.toggle-map {
      margin-top: 10px;
      background-color: #1976d2;
      color: white;
      border: none;
      padding: 8px 16px;
      border-radius: 4px;
      cursor: pointer;
    }

    button.toggle-map:hover {
      background-color: #125ea5;
    }

    .page-loader {
      position: fixed;
      top: 0;
      left: 0;
      width: 100%;
      height: 100%;
      background: rgba(0, 0, 0, 0.7);
      z-index: 9999;
      display: flex;
      align-items: center;
      justify-content: center;
    }

    .page-loader .loader-bar {
      text-align: center;
      width: 200px;
      color: #fff;
    }

    .page-loader .bar {
      width: 100%;
      height: 10px;
      background: #2e7d32;
      position: relative;
      overflow: hidden;
      border-radius: 4px;
    }

    .page-loader .bar::before {
      content: "";
      position: absolute;
      top: 0;
      left: -40%;
      width: 40%;
      height: 100%;
      background: linear-gradient(to right, #81c784, #2e7d32, #81c784);
      animation: slide 1s infinite ease-in-out;
      border-radius: 4px;
    }

    @keyframes slide {
      0% { left: -40%; }
      50% { left: 100%; }
      100% { left: 100%; }
    }
  </style>
</head>
<body>
  <div id="page-loader" class="page-loader" style="display: none;">
    <div class="loader-bar">
      <div class="bar"></div>
      <p>rendering..</p>
    </div>
  </div>

  <h1>Trace</h1>

  <form method="POST" onsubmit="return showPageLoader(this)">
    <input type="text" name="location" placeholder="search for a location" required><br>
    from: <input type="date" name="start_date" required>
    to: <input type="date" name="end_date" required><br>

    <label for="interval">image sampling interval:</label>
    <select name="interval" id="interval">
      <option value="1">daily</option>
      <option value="3" selected>every three days</option>
      <option value="5">every five days</option>
      <option value="7">weekly</option>
    </select>

    <input type="submit" name="action" value="go">
  </form>

  <div class="map-container">
    <figure style="margin: 0;">
      {{ map_html|safe }}
      <figcaption style="font-size: 13px; color: #ccc; text-align: center; margin-top: 8px;">
        this map depicts the average vegetation index values across the input time frame
      </figcaption>
    </figure>
  </div>

  {% if image_url and flipbook_frames and flipbook_frames|length > 0 %}
  <div class="content-wrapper">
    <div class="card">
      <h2>Greenness Index</h2>
      <img src="{{ url_for('static', filename=image_url) }}" width="700">
    </div>

    <div class="card">
      <h2>Flipbook</h2>
      <div id="imageMode">
        <img id="ndvi_frame" src="{{ url_for('static', filename=flipbook_frames[0]) }}">
        <input type="range" id="slider" min="0" max="{{ flipbook_frames|length - 1 }}" value="0">
        <p id="frame_date"></p>
      </div>

      <div id="mapMode">
        {{ mini_map_html|safe }}
      </div>

      <button class="toggle-map" onclick="toggleMap()">toggle base map</button>
    </div>
  </div>

  <script>
    const frames = {{ flipbook_frames|tojson }};
    const img = document.getElementById("ndvi_frame");
    const slider = document.getElementById("slider");
    const label = document.getElementById("frame_date");
    const imageMode = document.getElementById("imageMode");
    const mapMode = document.getElementById("mapMode");

    function update(index) {
      img.src = "/static/" + frames[index];
      label.textContent = "Date: " + frames[index].split("_")[1].split(".")[0];
    }

    function toggleMap() {
      if (imageMode.style.display === "none") {
        imageMode.style.display = "block";
        mapMode.style.display = "none";
        update(slider.value);
      } else {
        imageMode.style.display = "none";
        mapMode.style.display = "block";
        label.textContent = "Interactive Map View";
      }
    }

    slider.addEventListener("input", () => update(slider.value));
    update(0);
  </script>
  {% endif %}

  <script>
    function showPageLoader(form) {
      const loader = document.getElementById("page-loader");
      if (loader) {
        loader.style.display = "flex";
      }
      setTimeout(() => form.submit(), 50);
      return false;
    }
  </script>
</body>
</html>
"""

In [7]:
# Web & Server
from flask import Flask, render_template_string, request
from pyngrok import ngrok
import requests

# Geospatial & Mapping
import ee
import folium
import geojson
from geopy.geocoders import Nominatim

# Data & ML
import joblib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# System & Utilities
import os
import threading
import time
import json
import uuid
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

STATIC_DIR = "static"
os.makedirs(STATIC_DIR, exist_ok=True)
app = Flask(__name__)
port = 7000

ndvi_palette = ['#0000cc', 'brown', '#6b0303', 'orange', 'green', 'darkgreen']
vis_params = {'min': -1, 'max': 1, 'palette': ndvi_palette}
blend_params = {'min': 0, 'max': 1, 'palette': ndvi_palette}
HOOKS = {}

def register_hook(name, fn):
    HOOKS[name] = fn

def run_hook(name, data):
    fn = HOOKS.get(name)
    return fn(data) if fn else data

# Hook to detect anomalies over time ( tracks list of timestamps where change > mean + 2*std )
def anomaly_detect(ndvi_data: dict):
    means = np.array(ndvi_data["ndvi_means"])
    diffs = np.abs(np.diff(means))
    threshold = np.nanmean(diffs) + 2 * np.nanstd(diffs)
    anomalies = [
        ndvi_data["timestamps"][i+1]
        for i, d in enumerate(diffs)
        if d > threshold
    ]
    ndvi_data["anomalies"] = anomalies
    return ndvi_data

register_hook("anomaly_detect", anomaly_detect)


# Hook to export anomaly data as GeoJSON file
def geojson_export(ndvi_data: dict):
    features = []
    for ts in ndvi_data.get("anomalies", []):
        features.append(geojson.Feature(
            geometry=geojson.Point(ndvi_data["center"]),
            properties={"timestamp": ts}
        ))
    with open("ndvi_anomalies.geojson", "w") as f:
        geojson.dump(geojson.FeatureCollection(features), f)
    return ndvi_data

register_hook("geojson_export", geojson_export)


# Hook to log mean pixel value of each saved frame to frame_stats.json
def frame_logger(image_path: str):
    img = Image.open(image_path)
    arr = np.array(img).astype(float) / 255.0
    stats = {
        "frame": image_path,
        "timestamp": image_path.split("_")[1].split(".")[0],
        "mean_pixel": float(np.nanmean(arr))
    }
    with open("frame_stats.json", "a") as f:
        f.write(json.dumps(stats) + "\n")
    return image_path

register_hook("frame_logger", frame_logger)


# # Hook to forecast vegetation index values using a pre-trained model
""" model = joblib.load("ndvi_forecast_model.pkl") # Uncomment when valid model is available

def ml_model(ndvi_data: dict):
    X = np.array(ndvi_data["ndvi_means"]).reshape(-1, 1)
    preds = model.predict(X)
    ndvi_data["forecast"] = preds.tolist()
    return ndvi_data

register_hook("ml_model", ml_model) """

' model = joblib.load("ndvi_forecast_model.pkl") # Uncomment when valid model is available\n\ndef ml_model(ndvi_data: dict):\n    X = np.array(ndvi_data["ndvi_means"]).reshape(-1, 1)\n    preds = model.predict(X)\n    ndvi_data["forecast"] = preds.tolist()\n    return ndvi_data\n\nregister_hook("ml_model", ml_model) '

In [8]:
# Web & Server
from flask import Flask, render_template_string, request
from pyngrok import ngrok
import requests

# Geospatial & Mapping
import ee
import folium
import geojson
from geopy.geocoders import Nominatim

# Data & ML
import joblib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# System & Utilities
import os
import threading
import time
import json
import uuid
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

STATIC_DIR = "static"
os.makedirs(STATIC_DIR, exist_ok=True)
app = Flask(__name__)
port = 7000

ndvi_palette = ['#0000cc', 'brown', '#6b0303', 'orange', 'green', 'darkgreen']
vis_params = {'min': -1, 'max': 1, 'palette': ndvi_palette}
blend_params = {'min': 0, 'max': 1, 'palette': ndvi_palette}

def geocode_location(location_name):
    geo = Nominatim(user_agent="ndvi-search")
    loc = geo.geocode(location_name)
    if loc:
        return [loc.latitude, loc.longitude]
    return [0, 0]


def get_modis_timeseries(ee_point, start_date, end_date):
    modis_ic = ee.ImageCollection("MODIS/061/MOD13A2") \
        .filterBounds(ee_point) \
        .filterDate(start_date, end_date) \
        .select("NDVI") \
        .sort("system:time_start") \
        .map(lambda img: img.set({
            'date': img.date().format('YYYY-MM-dd'),
            'mean': img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=ee_point,
                scale=250
            ).get('NDVI')
        }))
    features = modis_ic.getInfo()['features']
    dates, values = [], []
    for f in features:
        props = f['properties']
        if props.get('mean') is not None:
            dates.append(datetime.strptime(props['date'], "%Y-%m-%d"))
            values.append(round(props['mean'] / 10000, 3))
    return dates, values


def get_sentinel_timeseries(ee_point, start_date, end_date):
    def compute_s2_ndvi(img):
        ndvi = img.expression(
            '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
            {
                'B8': img.select('B8'),
                'B4': img.select('B4'),
                'B2': img.select('B2')
            }).rename('NDVI')
        return img.addBands(ndvi).set({
            'date': img.date().format('YYYY-MM-dd'),
            'mean': ndvi.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=ee_point,
                scale=30
            ).get('NDVI')
        })

    s2_ic = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterBounds(ee_point) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select(['B2', 'B4', 'B8']) \
        .map(compute_s2_ndvi) \
        .sort("system:time_start")

    features = s2_ic.getInfo()['features']
    dates, values = [], []
    for f in features:
        props = f['properties']
        if props.get('mean') is not None:
            dates.append(datetime.strptime(props['date'], "%Y-%m-%d"))
            values.append(round(props['mean'], 3))
    return dates, values


def generate_ndvi_chart(modis_dates, modis_values, s2_dates, s2_values, location_name, start_date, end_date):
    filename = f"ndvi_{location_name.replace(' ', '_')}_{start_date}_{end_date}.png"
    out_path = os.path.join(STATIC_DIR, filename)
    plt.figure(figsize=(12, 5))
    plt.plot(modis_dates, modis_values, marker='o', linestyle='-', label='MODIS', color='forestgreen')
    plt.plot(s2_dates, s2_values, marker='x', linestyle='-', label='Sentinel-2', color='royalblue')
    plt.gcf().autofmt_xdate()
    plt.title(f"comparative Chart for {location_name}")
    plt.xlabel("date")
    plt.ylabel("vegetation index (MODIS: NDVI, Sentinel-2: EVI)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path, dpi=150, bbox_inches='tight')
    plt.close()
    return filename


def generate_ndvi_map(center, ee_point, start_date, end_date):
    m = folium.Map(location=center, zoom_start=10)
    modis_ndvi = ee.ImageCollection("MODIS/061/MOD13A2") \
        .filterBounds(ee_point) \
        .filterDate(start_date, end_date) \
        .select('NDVI') \
        .median() \
        .divide(10000).unitScale(0, 1).rename("NDVI")

    sentinel_ic = ee.ImageCollection("COPERNICUS/S2_SR") \
        .filterBounds(ee_point) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.listContains('system:band_names', 'B2')) \
        .filter(ee.Filter.listContains('system:band_names', 'B4')) \
        .filter(ee.Filter.listContains('system:band_names', 'B8')) \
        .select(['B2', 'B4', 'B8'])

    ndvi_s2 = None
    if sentinel_ic.size().getInfo() > 0:
        sentinel = sentinel_ic.median().setDefaultProjection('EPSG:4326', None, 30)
        ndvi_s2 = sentinel.expression(
            '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
            {
                'B8': sentinel.select('B8'),
                'B4': sentinel.select('B4'),
                'B2': sentinel.select('B2')
            }
        ).unitScale(-1, 1).rename("NDVI")
        #water_mask = ee.Image("JRC/GSW1_3/GlobalSurfaceWater").select("occurrence").lt(95)
        ndvi_s2 = ndvi_s2.where(ndvi_s2.lt(0), -1)#.updateMask(water_mask)

    m.add_ee_layer(modis_ndvi, vis_params, 'MODIS NDVI')
    if ndvi_s2:
        #m.add_ee_layer(ndvi_s2, vis_params, 'Sentinel-2 NDVI')
        blended = modis_ndvi.add(ndvi_s2).divide(2).rename("Blended NDVI")
        m.add_ee_layer(blended, blend_params, 'Blended NDVI')

    folium.LayerControl().add_to(m)
    m.get_root().html.add_child(folium.Element(legend_html))
    return m


def generate_flipbook_frames(ee_point, start_date, end_date, interval_days, max_workers=4):
    roi_buffer = ee_point.buffer(2500).bounds()
    current = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    flipbook_paths = []

    # Updated palette: red/yellow for low NDVI, green only for healthy vegetation
    vis_params = {
        'min': 0,
        'max': 0.8,
        'palette': ['#d73027', '#fdae61', '#ffffbf', '#a6d96a', '#1a9850']
    }

    def process_frame(date):
        try:
            s2 = ee.ImageCollection("COPERNICUS/S2_SR") \
                .filterBounds(roi_buffer) \
                .filterDate(date.strftime('%Y-%m-%d'), (date + timedelta(days=2)).strftime('%Y-%m-%d')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .median() \
                .select(['B2', 'B4', 'B8'])

            if s2.bandNames().size().getInfo() == 3:
                ndvi = s2.expression(
                    '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
                    {
                        'B8': s2.select('B8'),
                        'B4': s2.select('B4'),
                        'B2': s2.select('B2')
                    }
                ).unitScale(0, 0.8).visualize(**vis_params)

                thumb_params = {
                    'dimensions': 300,
                    'region': roi_buffer.getInfo(),
                    'format': 'png'
                }
                url = ndvi.getThumbURL(thumb_params)
                filename = f"ndvi_{date.strftime('%Y-%m-%d')}.png"
                out_path = os.path.join(STATIC_DIR, filename)
                with open(out_path, 'wb') as f:
                    f.write(requests.get(url).content)
                return filename
        except Exception as e:
            print(f"⚠️ Skipped {date.strftime('%Y-%m-%d')}: {e}")
        return None

    # Generate list of dates
    dates = []
    while current <= end_dt:
        dates.append(current)
        current += timedelta(days=interval_days)

    # Run in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(process_frame, dates))

    return [r for r in results if r]

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


def cleanup_old_images(folder=STATIC_DIR, max_age_seconds=86400):
    now = time.time()
    for fname in os.listdir(folder):
        fpath = os.path.join(folder, fname)
        if fname.endswith('.png') and os.path.isfile(fpath):
            if now - os.path.getmtime(fpath) > max_age_seconds:
                os.remove(fpath)

In [ ]:
# Web & Server
from flask import Flask, render_template_string, request
from pyngrok import ngrok
import requests

# Geospatial & Mapping
import ee
import folium
import geojson
from geopy.geocoders import Nominatim

# Data & ML
import joblib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# System & Utilities
import os
import threading
import time
import json
import uuid
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

ee.Initialize(project='your_project_id')

# Define visualisation palettes
ndvi_palette = ['#0000cc', 'brown', '#6b0303', 'orange', 'green', 'darkgreen']
vis_params = {'min': -1, 'max': 1, 'palette': ndvi_palette}
blend_params = {'min': 0, 'max': 1, 'palette': ndvi_palette}

legend_html = """
<div style="position: fixed;
     bottom: 30px; left: 30px; width: 180px; height: 160px;
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; padding: 10px;">
<b>Colour Key</b><br>
<i style="background:#0000cc;width:18px;height:18px;float:left;margin-right:8px;"></i> Water/Snow<br>
<i style="background:brown;width:18px;height:18px;float:left;margin-right:8px;"></i> Bare Soil<br>
<i style="background:#6b0303;width:18px;height:18px;float:left;margin-right:8px;"></i> Sparse Veg<br>
<i style="background:orange;width:18px;height:18px;float:left;margin-right:8px;"></i> Moderate Veg<br>
<i style="background:green;width:18px;height:18px;float:left;margin-right:8px;"></i> Dense Veg<br>
<i style="background:darkgreen;width:18px;height:18px;float:left;margin-right:8px;"></i> Very Dense Veg
</div>
"""

# App setup
STATIC_DIR = "static"
os.makedirs(STATIC_DIR, exist_ok=True)
app = Flask(__name__)
port = 7000

# Main route
@app.route('/', methods=['GET', 'POST'])
def index():
    center = [0, 0]
    image_url = None
    flipbook_paths = []
    mini_map_html = ""

    if request.method == 'POST':
        # Get input
        location_name = request.form.get("location")
        start_date = request.form.get("start_date")
        end_date = request.form.get("end_date")
        interval_days = int(request.form.get("interval", 3))

        # Centre location
        center = geocode_location(location_name)
        ee_point = ee.Geometry.Point(center[::-1])

        # Clean out old images
        threading.Thread(target=cleanup_old_images).start()

        modis_dates, modis_values = get_modis_timeseries(ee_point, start_date, end_date)
        s2_dates, s2_values = get_sentinel_timeseries(ee_point, start_date, end_date)

        # VI chart
        image_url = generate_ndvi_chart(modis_dates, modis_values, s2_dates, s2_values,
                                        location_name, start_date, end_date)

        # Map with overlays
        m = generate_ndvi_map(center, ee_point, start_date, end_date)
        mini_map = folium.Map(location=center, zoom_start=10, control_scale=True, width="450px", height="450px")
        mini_map_html = mini_map._repr_html_()

        # Flipbook
        flipbook_paths = generate_flipbook_frames(ee_point, start_date, end_date, interval_days)

        # Run frame_logs hook
        for path in flipbook_paths:
            if os.path.exists(path):
              run_hook("frame_logger", path)
            else:
              print(f"Frame not found: {path}")

        # Bundle NDVI data
        ndvi_data = {
          "timestamps": [d.strftime("%Y%m%d") for d in modis_dates],  # or s2_dates
          "ndvi_means": modis_values,  # or s2_values
          "center": center
        }

        # Run hooks
        ndvi_data = run_hook("anomaly_detect", ndvi_data)
        ndvi_data = run_hook("geojson_export", ndvi_data)

        # Run hook to link model
        try:
            ndvi_data = run_hook("ml_model", ndvi_data)
        except Exception as e:
            print("Forecast model not loaded:", e)

        return render_template_string(TEMPL,
                                      map_html=m._repr_html_(),
                                      image_url=image_url,
                                      flipbook_frames=flipbook_paths,
                                      mini_map_html=mini_map_html)

    # GET fallback
    m = folium.Map(location=center, zoom_start=2)
    return render_template_string(TEMPL,
                                  map_html=m._repr_html_(),
                                  image_url=None,
                                  flipbook_frames=[],
                                  mini_map_html=mini_map_html)

if __name__ == '__main__':
    public_url = ngrok.connect(port)
    print(f"🔗 Public URL: {public_url}")
    app.run(port=port)

🔗 Public URL: NgrokTunnel: "https://89a86527df00.ngrok-free.app" -> "http://localhost:7000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:7000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Aug/2025 10:39:16] "HEAD /https://89a86527df00.ngrok-free.app/ HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Aug/2025 10:39:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Aug/2025 10:39:21] "GET /favicon.ico HTTP/1.1" 404 -


⚠️ Skipped 2025-01-01: Image.select: Band pattern 'B2' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
⚠️ Skipped 2025-01-08: Image.select: Band pattern 'B2' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
⚠️ Skipped 2025-02-12: Image.select: Band pattern 'B2' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
⚠️ Skipped 2025-02-19: Image.select: Band pattern 'B2' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
⚠️ Skipped 2025-02-26: Image.select: Band pattern 'B2' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
⚠️ Skipped 2025-03-19: Image.select: Band pattern 'B2' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-band

INFO:werkzeug:127.0.0.1 - - [19/Aug/2025 10:39:53] "POST / HTTP/1.1" 200 -


⚠️ Frame not found: ndvi_2025-01-15.png
⚠️ Frame not found: ndvi_2025-01-22.png
⚠️ Frame not found: ndvi_2025-01-29.png
⚠️ Frame not found: ndvi_2025-02-05.png
⚠️ Frame not found: ndvi_2025-03-05.png
⚠️ Frame not found: ndvi_2025-03-26.png
⚠️ Frame not found: ndvi_2025-04-02.png
⚠️ Frame not found: ndvi_2025-04-09.png
⚠️ Frame not found: ndvi_2025-04-16.png
⚠️ Frame not found: ndvi_2025-04-30.png
⚠️ Frame not found: ndvi_2025-05-07.png
⚠️ Frame not found: ndvi_2025-05-21.png
⚠️ Frame not found: ndvi_2025-06-04.png
⚠️ Frame not found: ndvi_2025-06-11.png
⚠️ Frame not found: ndvi_2025-06-25.png
⚠️ Frame not found: ndvi_2025-07-02.png
⚠️ Frame not found: ndvi_2025-07-16.png


INFO:werkzeug:127.0.0.1 - - [19/Aug/2025 10:39:54] "GET /static/ndvi_Abu_Dhabi_2025-01-01_2025-07-31.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Aug/2025 10:39:55] "GET /static/ndvi_2025-01-15.png HTTP/1.1" 200 -
